In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [4]:
type(x_train[0][0, 0, 0])

numpy.float64

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [15]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation="relu")
    self.flatten = Flatten()
    self.d1 = Dense(128, activation="relu")
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)

    return x

model = MyModel()

In [16]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [17]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [20]:
@tf.function
def train(images, labels):
  with tf.GradientTape() as tape:
    pred = model(images)
    loss = loss_function(labels, pred)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, pred)

In [21]:
@tf.function
def test(images, labels):
  pred = model(images)
  loss = loss_function(labels, pred)
  test_loss(loss)
  test_accuracy(labels, pred)

In [24]:
epochs = 5

for epoch in range(epochs):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  #train
  for images, labels in train_dataset:
    train(images, labels)

  #test
  for images, labels in test_dataset:
    test(images, labels)

    print("Epoch: ", epoch + 1,
          f"Train loss: {train_loss.result()}",
          f"Train accuracy: {train_accuracy.result()}",
          f"Test loss: {test_loss.result()}",
          f"test accuracy: {test_accuracy.result()}")

Epoch:  1 Train loss: 0.0049452162347733974 Train accuracy: 0.998449981212616 Test loss: 0.036743540316820145 test accuracy: 0.96875
Epoch:  1 Train loss: 0.0049452162347733974 Train accuracy: 0.998449981212616 Test loss: 0.018648695200681686 test accuracy: 0.984375
Epoch:  1 Train loss: 0.0049452162347733974 Train accuracy: 0.998449981212616 Test loss: 0.012478522956371307 test accuracy: 0.9895833134651184
Epoch:  1 Train loss: 0.0049452162347733974 Train accuracy: 0.998449981212616 Test loss: 0.017297983169555664 test accuracy: 0.984375
Epoch:  1 Train loss: 0.0049452162347733974 Train accuracy: 0.998449981212616 Test loss: 0.021953780204057693 test accuracy: 0.981249988079071
Epoch:  1 Train loss: 0.0049452162347733974 Train accuracy: 0.998449981212616 Test loss: 0.01829775609076023 test accuracy: 0.984375
Epoch:  1 Train loss: 0.0049452162347733974 Train accuracy: 0.998449981212616 Test loss: 0.04434599727392197 test accuracy: 0.9821428656578064
Epoch:  1 Train loss: 0.004945216234

In [30]:
import cv2
import numpy as np

img = cv2.imread("eight.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
img = img[..., tf.newaxis]
img = img[tf.newaxis, ...]
img = img.astype("float32")

# img.shape

pred = model(img)
result = np.argmax(pred)
result

8

In [31]:
model.save("/content/drive/MyDrive/models/my_mnist_model")

In [32]:
model = tf.keras.models.load_model("/content/drive/MyDrive/models/my_mnist_model")